In [127]:
import os
import re
import pandas as pd
import torch
import torch.nn as nn
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

# import torch.nn.functional as F
# import torch.optim as optim
# from tqdm import tqdm, tqdm_notebook
# from torch.utils.data import Dataset, DataLoader
# from datasets import load_dataset, load_metric, load_from_disk
# from transformers import TrainingArguments, Trainer
# from transformers import DataCollatorWithPadding, EarlyStoppingCallback, AdamW

# import random
# import numpy as np
# import gluonnlp as nlp
# from transformers.optimization import get_cosine_schedule_with_warmup
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model

In [4]:
df = pd.read_csv("data/train_report.csv")
df.head()

,filename,article,length,label
0,311.txt,"하지만 이번 경기 Cycle 에는 이자유예, 양극화 등 여러 요인으로 자산건전성이 ...",363,0
1,strongbuy465.txt,오직 우려되는 사항은 해외 플랜트 부문의 실적이 언제쯤 개선될지다. 이번 2Q16 ...,283,1
2,strongbuy164.txt,"투자의견 STRONG BUY 유지, 목표주가 146,000원으로 상향 한국항공우주 ...",397,1
3,보유하향59.txt,목표주가 하향조정하며 HOLD 투자의견 유지: 이익전망 미세조정에도 불구 목표주가는...,313,0
4,strongbuy301.txt,"‘the’ 기록적인 실적 3Q15 Review: 매출 7,040억원, 영업이익 79...",1203,1


In [128]:
checkpoint = "skt/kobert-base-v1"
model = AutoModel.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [129]:
print(model)
# print('-'*100)
# print(tokenizer)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )

In [120]:
sequence = df.iloc[:3]['article'].to_list()
print(f"length: {len(sequence)} \n{sequence}")
print()

# print("After kobert_tokenize")
# tokens = tokenizer.tokenize(sequence)
# print(f"length: {len(tokens)} \n{tokens}"); print()

# print("After Convert tokens to ids")
# ids = tokenizer.convert_tokens_to_ids(tokens)
# print(f"length: {len(ids)} \n{ids}"); print()

# print("After convert torch.tensor")
# input_ids = torch.tensor([ids])
# print(f"length: {len(input_ids)} \n{input_ids}")

# 위와 동일한 결과
tokens = tokenizer(sequence, padding=True, max_length=512, truncation=True, return_tensors='pt')
print(tokens)

length: 3 
['하지만 이번 경기 Cycle 에는 이자유예, 양극화 등 여러 요인으로 자산건전성이 경기에 후행할 수도 있다. 2021 년 순이익 추정치를 1 조 5,560 억원으로 6% 상향조정하지만, 여느 때에 비해 실적의 불확실성은 높은 편이다. 장기적인 배당 불확실성 감안하여 목표주가 9,000 원으로 하향조정기업은행은 국책은행이어서 금감원의 배당 축소 권고 대상에 포함되지 않는다. 그럼에도 불구하고 과연 배당성향을 더 높일 수 있을지는 불확실하다. 앞으로도 정책적인 역할을 중시할 가능성이 있기 때문이다. 장기 배당성향 기대치를 35%에서 30%로 낮추면서목표주가를 10,000 원에서 9,000 원으로 하향조정한다. 기존의 ‘중립’ 의견을 유지한다.', '오직 우려되는 사항은 해외 플랜트 부문의 실적이 언제쯤 개선될지다. 이번 2Q16 실적에서도 대규모 손실이 인식된 것과 같이 해외 상황은 여전히 녹록하지 않기 때문이다. 하지만 이런 기간은 머지않아 종료될 것으로 판단된다. 이미 해외공사들이 종료단계에 들어섰으며 2016년 내로 대부분 완료될 것으로 판단되기 때문에, 2017년을 내다볼 경우 단연 GS건설의 매수가 필요한 시점이다. 2017년 실적 추정치기준(희석 반영 후) PER 9.4배, PBR 0.6배에 그치는 현재 주가는 충분히 매력이 있다.', '투자의견 STRONG BUY 유지, 목표주가 146,000원으로 상향 한국항공우주 투자의견 STRONG BUY를 유지하며, 목표주가는 기존 115,000원에서 27.0% 상향 조정한 146,000원으로 수정 제시한다. 목표주가 상향근거는 ROE 상향(21.2%\uf0e022.7%)에 따른 적용 밸류에이션 상향(PBR 5.3 -> 6.8배)에 기인한다. 2Q16 실적을 통해 연간 두 자리대의 영업이익률을 충분히 기록할 수 있음이 확인되었기 때문에 가능한 목표주가 상향이다. 2011년 상장 이후 짝수년도의 신규수주는 홀수년도 대비 부진했다. 현재는 실적 증가에 관심을 둘 때일 뿐, 수주는 2017년 

In [122]:
output = model(tokens)
print(output)

AttributeError: 